# Final Project: Predicting Covid Positive From Vaccines

In [1]:
from utils import *

## 1. Ask: Research Question / Hypothesis
----
#### **Research Question(s):** Given demographic information, which classification model most accurately predicts whether not someone will get Covid after getting the vaccine?  


#### **Hypothesis:** I believe the random forest classifier would provide the most accurate estimation for this classification problem since it's robust to outliers.


----

## 2. Acquire Data
----
#### **Data**: CDC's Vaccine Adverse Event Reporting System
1. Data Variables 
    - **id**: Unique Patient ID
    - **manu**: makers of vaccine products
    - **dose**: the number of vaccine doses given for a vaccine
    - **route**: route of administration/ the path the vaccine was taken into the body
    - **site**: the site for a vaccine product
    - **recovd_date:** indicates when the patient recovered from adverse symptoms
    - **state**: Patient's Home State
    - **age**: Patient's Age
    - **sex**: Patient's Sex
    - **hospdays**: Days spent in the hospital recovering from adverse symptoms
    - **disable**: Whether or not patient is disabled
    - **recvd**: Whether or not patient recovered from adverse symptoms
    - **numdays**: Number of days patient had adverse symptoms
    - **adminby**: public, private, other, military, work, pharmacy, senior living, school, unknown agency
    - **hosp_visit**: Whether or not patient had to visit the hospital from adverse symptoms
    - **er_visit**: Whether or not patient had to visit the ER from adverse symptoms
    - **history**: Patient's istory of diseases
    - **allergies**: Patient's allergies
    - **other_meds**: Patient's current medications
    - **symptom1** 1st adverse symptom from vaccine
    - **symptom2**: 2nd adverse symptom from vaccine
    - **symptom3**: 3rd adverse symptom from vaccine
    - **symptom4**: 4th adverse symptom from vaccine
    - **symptom5**: 5th adverse symptom from vaccine

In [55]:
df = pd.read_csv('vaccine_df.csv')

In [56]:
df

,id,manu,dose,route,state,age,sex,hospdays,disable,recvd,...,other_meds,history,hosp_visit,er_visit,allergies,symptom1,symptom2,symptom3,symptom4,symptom5
0,916600,MODERNA,1,IM,TX,33.0,F,NaN,NaN,Y,...,None,None,Y,NaN,Pcn and bee venom,Dysphagia,Epiglottitis,NaN,NaN,NaN
1,916601,MODERNA,1,IM,CA,73.0,F,NaN,NaN,Y,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Y,NaN,"""Dairy""",Anxiety,Dyspnoea,NaN,NaN,NaN
2,916602,PFIZER\BIONTECH,1,IM,WA,23.0,F,NaN,NaN,U,...,None,None,NaN,Y,Shellfish,Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,NaN
3,916604,MODERNA,1,IM,TX,47.0,F,NaN,NaN,N,...,Na,NaN,NaN,NaN,Na,Injection site erythema,Injection site pruritus,Injection site swelling,Injection site warmth,NaN
4,916606,MODERNA,1,IM,NV,44.0,F,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,iodine (shellfish) has epipen,Pharyngeal swelling,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18107,1057082,PFIZER\BIONTECH,1,IM,KY,86.0,M,12.0,NaN,N,...,"Meds prior to admission: Amiodarone, ASA, Lipi...","Arthritis, Afib, chronic anticoagulation, chro...",NaN,NaN,Pneumococcal vaccine - rash,Malaise,Nausea,Oxygen therapy,Parosmia,Productive cough
18108,1057082,PFIZER\BIONTECH,1,IM,KY,86.0,M,12.0,NaN,N,...,"Meds prior to admission: Amiodarone, ASA, Lipi...","Arthritis, Afib, chronic anticoagulation, chro...",NaN,NaN,Pneumococcal vaccine - rash,SARS-CoV-2 test positive,Streptococcus test negative,Taste disorder,Vomiting,NaN
18109,1057281,MODERNA,1,IM,MD,77.0,F,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Death,NaN,NaN,NaN,NaN
18110,1057348,PFIZER\BIONTECH,1,IM,TX,88.0,F,NaN,NaN,N,...,"lasik, blood pressure",congestive heart failure,Y,NaN,none,Death,Dysarthria,Dysstasia,Fatigue,Feeding disorder


## 3. Process Data
----

1. **Fill Specific NA Columns with Appropriate Values** 
    - hospdays: 0.0
    - disable: 'N'
    - hosp_visit: 'N'
    - er_visit: 'N'


2. **Merge the Symptoms into Singular Column** 
    - Create a new column that holds a flattened list of all symptoms 
    
    
3. **Create a new boolean column for every top allergy, other diagnosis, and medication**

   
4. **Create 'Covid' Target Column** 
    - There are two symptoms that encompass whether or not someone got covid 'SARS-CoV-2 test positive' and'COVID-19'. 
    - Need to merge those two columns together and return True if either of them are True
    
    
5. **Drop 'SARS-CoV-2 test positive', 'COVID-19' and 'All Symptoms' Columns**
    

In [57]:
pipe_symp = Pipeline([('fillNaNs',            FillNaNs(list(fillNa.keys()), list(fillNa.values()))),             # Fill specific NA columns 
                      ('Merge Symptoms',      MergeSymptomDFs(symptoms, 'id', final_cols[:2], final_cols[1:])),  # Merge the symptoms into singular column
                      ('Expand Symptoms',     ConvertColumnLists(covid_cols[:2], final_cols[0])),                # Create a new boolean column for covid-19/cov test pos
                      ('Expand History',      ConvertColumnLists(history_list, 'history')),                      # Create a new boolean column for every top symptom
                      ('Expand Meds',         ConvertColumnLists(meds, 'other_meds')),                           # Create a new boolean column for every top medication
                      ('Expand Allerigies',   ConvertColumnLists(meds, 'allergies')),                            # Create a new boolean column for every top allergy
                      ('Create Target',       BooleanInAnyDF(covid_cols[:2], True, covid_cols[-1])),             # Create 'covid' target column
                      ('Drop Columns',        DropColumns(covid_cols[:3] + text)),                               # Drop unnecessary columns
                     ])

df = pipe_symp.fit_transform(df)

In [58]:
df

,manu,dose,route,state,age,sex,disable,asthma,arthritis,diabetes,...,gerd,high blood pressure,depression,levothyroxine,multivitamin,tylenol,adderall,ibuprofen,naltrexone,COVID
0,MODERNA,1,IM,TX,33.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,MODERNA,1,IM,CA,73.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,PFIZER\BIONTECH,1,IM,WA,23.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,MODERNA,1,IM,TX,47.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,MODERNA,1,IM,NV,44.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38157,PFIZER\BIONTECH,1,IM,KY,86.0,M,N,False,True,False,...,True,False,False,False,False,False,False,False,False,True
38158,PFIZER\BIONTECH,1,IM,KY,86.0,M,N,False,True,False,...,True,False,False,False,False,False,False,False,False,True
38159,MODERNA,1,IM,MD,77.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
38160,PFIZER\BIONTECH,1,IM,TX,88.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [59]:
X = df.loc[:, (df.columns != 'COVID')]
y = df['COVID']

In [60]:
X

,manu,dose,route,state,age,sex,disable,asthma,arthritis,diabetes,...,hypothyroidism,gerd,high blood pressure,depression,levothyroxine,multivitamin,tylenol,adderall,ibuprofen,naltrexone
0,MODERNA,1,IM,TX,33.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,MODERNA,1,IM,CA,73.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,PFIZER\BIONTECH,1,IM,WA,23.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,MODERNA,1,IM,TX,47.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,MODERNA,1,IM,NV,44.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38157,PFIZER\BIONTECH,1,IM,KY,86.0,M,N,False,True,False,...,False,True,False,False,False,False,False,False,False,False
38158,PFIZER\BIONTECH,1,IM,KY,86.0,M,N,False,True,False,...,False,True,False,False,False,False,False,False,False,False
38159,MODERNA,1,IM,MD,77.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False
38160,PFIZER\BIONTECH,1,IM,TX,88.0,F,N,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [61]:
y.value_counts(normalize=True)

False    0.982103
True     0.017897
Name: COVID, dtype: float64

In [62]:
con_pipe = Pipeline([('imputer', SimpleImputer(strategy='median', add_indicator=True)), # replaces the NaN values with median
                     ('scaler', StandardScaler()),                                      # standardize the numerical features
                     ('var threshold', VarianceThreshold(0.06))]) 

cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), # replaces the NaN values with most frequent
                     ('encoder', OneHotEncoder(sparse=False)),             # represent categorical variables as binary vectors
                     ('var threshold', VarianceThreshold(0.06))]) 

final_prep_pipe = ColumnTransformer([('categorical', cat_pipe,   (X.dtypes == object)), 
                                    ('continuous',  con_pipe, ~(X.dtypes == object))])


In [63]:
X = final_prep_pipe.fit_transform(X)
y=y.values

In [64]:
X.shape

(38162, 25)

In [65]:
X

array([[ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 0.        ,  1.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       ...,
       [ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 0.        ,  1.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602]])

### Ensure the Same Ratio of Classes for Train, Val and Test

In [66]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 

for train_idx, test_idx in stratSplit.split(X, y): #ensures same underlying class distribution for train/test df
    X_train=X[train_idx]
    y_train=y[train_idx]
    X_test=X[test_idx]
    y_test=y[test_idx]

## 4. Model 
----
#### **Balanced Accuracy:** Since we have imbalanced data, balanced accuracy score is best to account for both covid positive and not covid positive outcome classes.

### Without Oversampling

In [67]:
estimators = [('KNeighbors            ', KNeighborsClassifier()),
              ('RF Classifier         ', RandomForestClassifier()),
              ('SGD Classifier        ', SGDClassifier()),
              ('Extra Trees Classifier', ExtraTreesClassifier()),
              ('AdaBoost Classifier   ', AdaBoostClassifier()),
              ('MLP Classifier        ', MLPClassifier(max_iter=500)),
              ('Logistic Classifier   ', LogisticRegression(max_iter=1000))]

for estimator in estimators:
    pipe = Pipeline([('Classifier', estimator[1])])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    print(f"{estimator[0]}: {round(balanced_accuracy_score(y_val, y_pred),3)}\n")


KNeighbors            : 0.545

RF Classifier         : 0.577

SGD Classifier        : 0.5

Extra Trees Classifier: 0.568

AdaBoost Classifier   : 0.5

MLP Classifier        : 0.514

Logistic Classifier   : 0.5



### SMOTE Oversampling

In [68]:
smote = SMOTE(sampling_strategy = 0.4) # duplicate instances of minority class
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [69]:
estimators = [('KNeighbors            ', KNeighborsClassifier()),
              ('RF Classifier         ', RandomForestClassifier()),
              ('SGD Classifier        ', SGDClassifier()),
              ('Extra Trees Classifier', ExtraTreesClassifier()),
              ('AdaBoost Classifier   ', AdaBoostClassifier()),
              ('MLP Classifier        ', MLPClassifier(max_iter=500)),
              ('Logistic Classifier   ', LogisticRegression(max_iter=1000))]

for estimator in estimators:
    pipe = Pipeline([('Classifier', estimator[1])])
    pipe.fit(X_smote, y_smote)
    y_pred = pipe.predict(X_val)
    print(f"{estimator[0]}: {round(balanced_accuracy_score(y_val, y_pred),3)}\n")

KNeighbors            : 0.607

RF Classifier         : 0.843

SGD Classifier        : 0.533

Extra Trees Classifier: 0.843

AdaBoost Classifier   : 0.591

MLP Classifier        : 0.743

Logistic Classifier   : 0.606



### Random Oversampling

In [70]:
X_train

array([[ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 0.        ,  1.        ,  0.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 0.        ,  1.        ,  0.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       ...,
       [ 1.        ,  0.        ,  1.        , ..., -0.06728675,
        -0.0461199 , -0.17940602],
       [ 0.        ,  1.        ,  1.        , ..., -0.06728675,
        -0.0461199 ,  5.57394901],
       [ 0.        ,  1.        ,  0.        , ..., -0.06728675,
        -0.0461199 , -0.17940602]])

In [71]:
oversample = RandomOverSampler(sampling_strategy = 0.4) # duplicate instances of minority class
X_oversample, y_oversample = oversample.fit_resample(X_train, y_train)

In [72]:
estimators = [('KNeighbors            ', KNeighborsClassifier()),
              ('RF Classifier         ', RandomForestClassifier()),
              ('SGD Classifier        ', SGDClassifier()),
              ('Extra Trees Classifier', ExtraTreesClassifier()),
              ('AdaBoost Classifier   ', AdaBoostClassifier()),
              ('MLP Classifier        ', MLPClassifier(max_iter=500)),
              ('Logistic Classifier   ', LogisticRegression(max_iter=1000))]

for estimator in estimators:
    pipe = Pipeline([('Classifier', estimator[1])])
    pipe.fit(X_oversample, y_oversample)
    y_pred = pipe.predict(X_val)
    print(f"{estimator[0]}: {round(balanced_accuracy_score(y_val, y_pred),3)}\n")


KNeighbors            : 0.613

RF Classifier         : 0.936

SGD Classifier        : 0.525

Extra Trees Classifier: 0.936

AdaBoost Classifier   : 0.631

MLP Classifier        : 0.733

Logistic Classifier   : 0.598



#### **Analysis 1:** Oversampling techniques improve model performance by providing more data from the minority class to learn from.
#### **Analysis 2:** Random Forest, Extra Trees, and MLP Classifiers performed the best.

In [73]:
estimators = {
    RandomForestClassifier: {
        'est__n_estimators': [100,300,500],
        'est__min_samples_leaf': [3,4,5]
    },
    ExtraTreesClassifier: {
        'est__n_estimators': [100,300,500],
        'est__min_samples_leaf': [3,4,5]
    },
    MLPClassifier: {
        'est__tol': [1e-3, 1e-4, 1e-5],
        'est__alpha': [1e-2, 1e-3, 1e-4],
        'est__max_iter':[1000]
    }
}

scores = []
for estimator, param_dist in estimators.items():

    pipe = Pipeline([('est', estimator())])

    rscv = RandomizedSearchCV(
        pipe,
        param_dist,
        n_jobs=-1,
    )
    rscv.fit(X, y)
    clean_best_param = {
        k.replace('est__', ''): v
        for k, v in rscv.best_params_.items()   
    }

    pipe = Pipeline([('est', estimator(**clean_best_param))])

    pipe.fit(X_oversample, y_oversample)
    y_pred = pipe.predict(X_val)
    score = balanced_accuracy_score(y_val, y_pred)
    scores.append((estimator.__name__, clean_best_param, score))

In [153]:
scores

[('RandomForestClassifier',
  {'n_estimators': 100, 'min_samples_leaf': 5},
  0.8470458470825627),
 ('ExtraTreesClassifier',
  {'n_estimators': 100, 'min_samples_leaf': 5},
  0.7933339758564297),
 ('MLPClassifier',
  {'tol': 0.001, 'max_iter': 1000, 'alpha': 0.01},
  0.8237826864502588)]

#### **Analysis:** Random Forest Performed Best

In [29]:
# additional hand hyperparamter tuning
pipe = Pipeline([('est', RandomForestClassifier(n_estimators=500,
                                                min_samples_leaf = 2,
                                                random_state=42))])

pipe.fit(X_r, y_oversample)
y_pred = pipe.predict(X_val)
print(f"RF Best Model: {round(balanced_accuracy_score(y_val, y_pred),3)}")

RF Best Model: 0.824


## 4. Deliver 
----
#### Test Set: Truest Measure of Generality

In [81]:
pipe = Pipeline([('est', RandomForestClassifier(n_estimators=500,
                                                min_samples_leaf = 2))])

pipe.fit(X_oversample, y_oversample)
y_pred = pipe.predict(X_test)
print(f"RF Best Model: {round(balanced_accuracy_score(y_test, y_pred),3)}\n")

RF Best Model: 0.778



## Summary
1. With ~ 82.4% balanced accuracy, we can predict whether or not someone who receives a vaccine will still get covid
2. The best model turned out to be random forest, possibly due to the fact that it is robust to outliers and feature’s that aren’t as important will simply not get chosen during feature selection
3. Despite class imbalance, we were able to get the model to perform fairly well due to oversampling techniques.
4. This is a useful model for those who may feel uncertain about getting the vaccine and wants to know their chances of getting covid even after receiving the vaccine

## Limitations 
1. Model only used the top allergies, diseases, and medication. The data pipeline dropped all others that may have had higher predictive power.
2. A lot of other variables that would be predictive of whether or not someone still gets covid is missing (e.g, whether or not someone lives in a city).
3. Next steps: Continue working on feature engineering. Carry out methods to measure feature importance and encode more allergies/diseases/medication. 

